In [ ]:
# -------------------------------
# Multi-seed, multi-class training for CIFAR-100
# -------------------------------

from lib.cifar100_utils import (
    set_seed, get_dataloaders, 
    build_cifar_resnet, build_wideresnet,
    train_model, load_model, evaluate_model
)
import os
import torch
import torch.nn as nn
import torch.optim as optim

# -------------------------------
# Configuration
# -------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 50
BATCH_SIZE = 512
NUM_CLASSES = 100
MODEL_DIR = "./../models/cifar100/"
DATASET_DIR = "./../data/"

# Define seeds and classes to remove
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]   # example seeds
REMOVE_CLASSES = [
    None, 4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100

MODEL_NAME = "resnet18" # or wideresnet

# Ensure model directory exists
os.makedirs(MODEL_DIR, exist_ok=True)

# -------------------------------
# Run experiments
# -------------------------------
for remove_class in REMOVE_CLASSES:
    for seed in SEEDS:
        print(f"\n=== Training: model={MODEL_NAME}, remove_class={remove_class}, seed={seed} ===")
        
        # Set seed
        set_seed(seed)
        
        # Data loaders
        trainloader, testloader = get_dataloaders(
            batch_size=BATCH_SIZE,
            remove_class=remove_class,
            dataset_dir=DATASET_DIR
        )
        
        # Build model
       # Build model (choose based on MODEL_NAME)
        if MODEL_NAME == "resnet18":
            model = build_cifar_resnet(num_classes=NUM_CLASSES, device=DEVICE)
        elif MODEL_NAME == "wideresnet":
            model = build_wideresnet(num_classes=NUM_CLASSES, device=DEVICE)
        else:
            raise ValueError(f"Unknown model: {MODEL_NAME}")
        
        # Optimizer, scheduler, criterion
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
        
        # Model path
        model_name = f"cifar100_{MODEL_NAME}_s{seed}_e{EPOCHS}"
        if remove_class is not None:
            model_name += f"_r{remove_class}"
        MODEL_PATH = os.path.join(MODEL_DIR, model_name + ".pth")
        
        # Train if not already saved
        if os.path.exists(MODEL_PATH):
            print(f"Model already exists at {MODEL_PATH}, skipping training.")
            load_model(model, MODEL_PATH, device=DEVICE)
        else:
            print(f"Training model and saving to {MODEL_PATH}...")
            train_model(
                model, trainloader, criterion, optimizer,
                scheduler, epochs=EPOCHS, save_path=MODEL_PATH,
                device=DEVICE
            )
        
        # Final evaluation
        print(f"Final evaluation for remove_class={remove_class}, seed={seed}:")
        preds, targets, confs = evaluate_model(model, testloader, criterion, device=DEVICE)



=== Training: model=resnet18, remove_class=39, seed=42 ===
Training model and saving to ./../models/cifar100/cifar100_resnet18_s42_e50_r39.pth...
Epoch [1/50] Train Loss: 3.904 | Train Acc: 10.27%
Epoch [2/50] Train Loss: 3.188 | Train Acc: 21.38%
Epoch [3/50] Train Loss: 2.706 | Train Acc: 30.41%
Epoch [4/50] Train Loss: 2.329 | Train Acc: 37.91%
Epoch [5/50] Train Loss: 2.040 | Train Acc: 44.48%
Epoch [6/50] Train Loss: 1.822 | Train Acc: 49.53%
Epoch [7/50] Train Loss: 1.655 | Train Acc: 53.51%
Epoch [8/50] Train Loss: 1.495 | Train Acc: 57.20%
Epoch [9/50] Train Loss: 1.379 | Train Acc: 60.43%
Epoch [10/50] Train Loss: 1.276 | Train Acc: 62.97%
Epoch [11/50] Train Loss: 1.191 | Train Acc: 65.01%
Epoch [12/50] Train Loss: 1.103 | Train Acc: 67.53%
Epoch [13/50] Train Loss: 1.036 | Train Acc: 69.26%
Epoch [14/50] Train Loss: 0.952 | Train Acc: 71.46%
Epoch [15/50] Train Loss: 0.894 | Train Acc: 73.17%
Epoch [16/50] Train Loss: 0.831 | Train Acc: 74.95%
Epoch [17/50] Train Loss: 0.77